## Init and Run GMM

In [1]:
%%javascript
Jupyter.utils.load_extensions('tdb_ext/main')

<IPython.core.display.Javascript object>

In [2]:
#this sets the backend to jupyter/ipython that (i think) displays
#     images directly. anyway, it prevents the matplotlib framework
#     python error that is my least favorite thing eeeevvvveeeer.
%matplotlib notebook

import sys
sys.path.append("..")

import gmix_model as gmix
import numpy as np
import tdb as tdb
import tensorflow as tf
import matplotlib.pyplot as plt

import os
os.chdir('/Users/azane/GitRepo/spider')

In [3]:
#read in training and test data
s_x, s_t = gmix.get_xt_from_npz('data/toy_train.npz')
t_x, t_t = gmix.get_xt_from_npz('data/toy_train.npz')

In [4]:
#create gmm with data
gmm = gmix.GaussianMixtureModel(s_x, s_t, t_x, t_t,
                               numGaussianComponents=1, hiddenLayerSize=5,
                               learningRate=0.01)

In [33]:
%%capture
#d will be a dictionary of evaluated tensors under their standard name.
runTimes = 1000
d = gmm.train(iterations=runTimes, testBatchSize=1000,
          trainBatchSize=500, reportEvery=runTimes)

## Debugging

In [35]:
#d['lay1']

In [32]:
#%%capture
print 'grad_w1'
print d['grad_w1']
print 'grad_b1'
print d['grad_b1']
print 'grad_w2'
print d['grad_w2']
print 'grad_b2'
print d['grad_b2']
print 'grad_w3'
print d['grad_w3']
print 'grad_b3'
print d['grad_b3']

grad_w1
[[  4.09395398e-05  -6.47133056e-05  -4.35368656e-06  -8.85692716e-05
   -2.64662176e-05]]
grad_b1
[ -4.25357321e-05   6.75513147e-05   4.51122469e-06   9.17089710e-05
   2.77349882e-05]
grad_w2
[[ -2.36529203e-07   6.60038495e-08  -1.38497469e-07   2.25666136e-05
   -6.14279315e-06]
 [ -3.49482207e-07   1.02265524e-07  -2.53392727e-07   4.60007323e-05
   -1.20455416e-05]
 [  6.78772437e-07  -1.91106295e-07   4.14092682e-07  -6.90282177e-05
    1.86285724e-05]
 [  1.61231995e-07  -4.23111217e-08   6.53226451e-08  -6.87518150e-06
    2.28644080e-06]
 [ -4.81978020e-07   1.39711659e-07  -3.35534963e-07   5.93732912e-05
   -1.57050436e-05]]
grad_b2
[  7.19258480e-07  -2.02575549e-07   4.39496517e-07  -7.33305496e-05
   1.97826939e-05]
grad_w3
[[ 0.         -0.00101285  0.        ]
 [ 0.          0.00101406  0.        ]
 [ 0.         -0.0010126   0.        ]
 [ 0.          0.00099078  0.        ]
 [ 0.          0.0009699   0.        ]]
grad_b3
[ 0.         -0.00101441  0.        ]


In [ ]:
#%%capture
print 'calc_agg_grad_w1'
print d['calc_agg_grad_w1']
print 'calc_agg_grad_b1'
print d['calc_agg_grad_b1']
print 'calc_agg_grad_w2'
print d['calc_agg_grad_w2']
print 'calc_agg_grad_b2'
print d['calc_agg_grad_b2']
print 'calc_agg_grad_w3'
print d['calc_agg_grad_w3']
print 'calc_agg_grad_b3'
print d['calc_agg_grad_b3']

In [8]:
inSize = 300.
ins = np.expand_dims((np.arange(inSize)/inSize), 1) #get input 0-1

In [9]:
lRate = 1

np_grad1 =  np.tanh( np.dot(ins, d['calc_agg_grad_w1']*lRate)\
                    + d['calc_agg_grad_b1']*lRate)
np_grad2 =  np.tanh( np.dot(np_grad1, d['calc_agg_grad_w2']*lRate)\
                    + d['calc_agg_grad_b2']*lRate)
np_grad3 =  np.tanh( np.dot(np_grad2, d['calc_agg_grad_w3']*lRate)\
                    + d['calc_agg_grad_b3']*lRate)

#np_grad1 =  np.dot(grad_ins, d['grad_w1']*lRate)# + np.tanh(d['grad_b1']*lRate)
#np_grad2 =  np.dot(np_grad1, d['grad_w2']*lRate)# + np.tanh(d['grad_b2']*lRate)
#np_grad3 =  np.dot(np_grad2, d['grad_w3']*lRate)# + np.tanh(d['grad_b3']*lRate)

#print np_grad3
#plt.plot(ins, np_grad3)

In [10]:
s = 200  # sample index for testing

In [11]:
%%capture
print 'netIn:'
print d['netIn'][s]

small_change = d['netIn'][s] + 0.05
print 'small_change:'
print small_change


In [12]:
%%capture
print 'w1:'
print d['w1']
print 'b1:'
print d['b1']
print 'lay1:'
print d['lay1'][s]


np_calc1 =  np.tanh( np.dot(d['netIn'][s], d['w1']) + d['b1'])
np_dif1 =  np_calc1 - d['lay1'][s]

print 'numpy calc:'
print np_calc1
print 'numpy calc dif:'
print np_dif1

print '------------------------------'
np_calc_sc1 = np.tanh( np.dot(small_change, d['w1']) + d['b1'])
np_dif_sc1 = np_calc1 - np_calc_sc1

print 'numpy calc:'
print np_calc1
print 'numpy calc of small change:'
print np_calc_sc1
print 'numpy calc dif with small change:'
print np_dif_sc1
print '------------------------------'


In [13]:
%%capture
print 'w2:'
print d['w2']
print 'b2:'
print d['b2']
print 'lay2:'
print d['lay2'][s]


np_calc2 =  np.tanh( np.dot(np_calc1, d['w2']) + d['b2'])
np_dif2 =  np_calc2 - d['lay2'][s]

print 'numpy calc:'
print np_calc2
print 'numpy calc dif:'
print np_dif2


In [14]:
%%capture
print 'w3:'
print d['w3']
print 'b3:'
print d['b3']
print 'netOut:'
print d['netOut'][s]


np_calc3 =  np.tanh( np.dot(np_calc2, d['w3']) + d['b3'])
np_dif3 =  np_calc3 - d['netOut'][s]

print 'numpy calc:'
print np_calc3
print 'numpy calc dif:'
print np_dif3